![Hypernets](hypernets/resources/img/logo.png)



# Field Deployment  : step by step

## &#x2022;  Step 1 : Mechanical position
### a) Use a bubble level on the mast to ensure the verticality

![pantilt_level](hypernets/resources/img/pan_tilt_bubble.png)


### b) Move the instrument into a horizontal position and adjust tilt if necessary (instrument must be horizontal when 90 or -90 asked position)

Edit config : http://10.42.0.1:8888/edit/config_hypernets.ini

In [ ]:
from ipywidgets import HBox, VBox, FloatText, Button
from IPython.display import display

pan = FloatText(description="Pan :")
tilt = FloatText(description="Tilt :")

power = Button(description="Power Relay On")
move = Button(description="Move Pan-Tilt")

@power.on_click
def power_relay_on(_):
    from hypernets.scripts.relay_command import set_state_relay
    set_state_relay(2, "on")
    set_state_relay(3, "on")
    set_state_relay(4, "on") # pan-tilt is on relay4 at lov

@move.on_click
def move_pan_tilt(_):
    from hypernets.scripts.pan_tilt import move_to
    move_to(None, pan.value, tilt.value, verbose=False, wait=False)

display(HBox((VBox((power, move)), VBox((pan, tilt)))))

## &#x2022;  Step 2 : Yoctopuce
### (meteo / GPS)

In [ ]:
from ipywidgets import HBox, VBox, Button, Label
from IPython.display import display

gps_show = Button(description="Get GPS location")
meteo = Button(description="Get Meteo Data")

gps_lbl = Label(value="")
meteo_lbl = Label(value="")


@gps_show.on_click
def show_gps_coords(_):
    from hypernets.scripts.yocto_gps import get_gps
    gps_lbl.value = get_gps(return_float=False)
    
@meteo.on_click
def show_meteo(_):
    from hypernets.scripts.yocto_meteo import get_meteo
    v_meteo = "   ".join([str(val) + unit for val, unit in get_meteo()])
    meteo_lbl.value = v_meteo
    
display(VBox((HBox((gps_show, gps_lbl)), HBox((meteo, meteo_lbl)))))

Here, you should double check if the [GPS] latitude and longitude are good in your configuration file :
http://10.42.0.1:8888/edit/config_hypernets.ini


## &#x2022; Step 3 : Point to the Sun :

In [ ]:
from hypernets.scripts.pan_tilt import move_to
from hypernets.scripts.spa.spa_hypernets import spa_from_datetime
from ipywidgets import HBox, Button, Label
from IPython.display import display

# point_gps = Button(description="Point to the sun")
point_datetime = Button(description="Point to the Sun")

# sun_gps_lbl = Label(value="")
sun_datetime_lbl = Label(value="")

@point_datetime.on_click
def point_sun_datetime(_):
    azimuth_sun, zenith_sun = spa_from_datetime()
    sun_datetime_lbl.value = f"Sun position : {azimuth_sun}, {zenith_sun}"
    move_to(None, azimuth_sun, 180-zenith_sun, verbose=False, wait=False)

# display(VBox((HBox((point_gps, sun_gps_lbl)), HBox((point_datetime, sun_datetime_lbl)))))
display((HBox((point_datetime, sun_datetime_lbl))))

## &#x2022; Step 4 : Take a Picture 

In [ ]:
from hypernets.scripts.call_radiometer import take_picture
from IPython.display import Image
from ipywidgets import HBox, Button, Label
from IPython.display import display

power_inst = Button(description="Power on Instrument")
take_pic = Button(description="Take a picture")

output_lbl = Label(value="")

@power_inst.on_click
def power_relay_on(_):
    from hypernets.scripts.relay_command import set_state_relay
    set_state_relay(2, "on")

@take_pic.on_click
def take_picture_action(_):
    # stream = take_picture(return_stream=True)
    # pic_sun = Image(stream)
    if take_picture():
        output_lbl.value = "ok (see http://10.42.0.1:8888/tree/DATA)"
    else:
        output_lbl.value = "error"
        
    # display(pic_sun)

display(HBox((power_inst, take_pic, output_lbl)))

## &#x2022; Take Spectrum (in progress...)

In [ ]:
from ipywidgets import Combobox, ToggleButtons, Dropdown, VBox, HBox

radiometer = Dropdown(description='Radiometer', options=['VNIR', 'SWIR', 'BOTH'])
entrance = Dropdown(description='Entrance', options=['Radiance', 'Irradiance', 'Dark'])
IT_Vnir = Dropdown(description='IT_Vnir', options=tuple([0]+[pow(2, i) for i in range(16)]))
IT_Swir = Dropdown(description='IT_Swir', options=tuple([0]+[pow(2, i) for i in range(16)]))

out = HBox([VBox([radiometer, entrance]), VBox([IT_Vnir, IT_Swir])])
display(out)

In [ ]:
from hypernets.virtual.create_metadata import parse_config_metadata
print("=" * 31 + " METADATA PREVIEW " + "=" * 31)
print(parse_config_metadata())
print("=" * 80)